In [ ]:
import requests
import newspaper
from tqdm import tqdm

In [ ]:
class Item:
    def __init__(self, title):
        self.title = title
        self.datasources = []

    async def await_datasources(self):
        self.datasources = [res for ds in self.datasources if (res:=await ds) is not None]
    
    def __str__(self):
        return f'# {self.title}\n\n' + ''.join([str(ds) for ds in self.datasources])

    def __add__(self, other):
        self.datasources.append(other)
        return self

class DataSource:
    def __init__(self, title, text):
        self.title = title
        self.text = text
    
    def __str__(self):
        return f'## {self.title}\n\n{self.text}\n\n'


async def get_wiki_data(lang, name):
    wiki_url = f'https://{lang}.wikipedia.org/api/rest_v1/page/summary/{name}'
    response = requests.get(wiki_url)
    if response.status_code != 200:
        return None
    data = response.json()
    # datasource = DataSource(f"Wikipedia: {data.get('title']} ({data['description']})", data['extract'])
    datasource = DataSource(f"Wikipedia: {data.get('title', '')} ({data.get('description', '')})", data.get('extract', ''))
    return datasource


async def get_website_data(url):
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()
        datasource = DataSource(f'Website: {article.title}', article.text)
        return datasource
    except Exception as e:
        return None

async def get_osm_data(tags):
    return DataSource(f'OpenStreetMap:', f'```\n{tags}\n```')

In [ ]:

# Define the Overpass API URL
url = "http://overpass-api.de/api/interpreter"

longitude = 13.3777
latitude = 52.5162
distance = 100
max_items = 10

# Define your Overpass query - fetching amenities within a radius around a point
query = f"""
[out:json];
(
  node["amenity"](around:{distance},{latitude},{longitude});
  way["amenity"](around:{distance},{latitude},{longitude});
  relation["amenity"](around:{distance},{latitude},{longitude});
);
out body;
>;
out skel qt;
"""

items = []

# Send the request to the Overpass API
response = requests.get(url, params={'data': query})

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    for elem in data['elements']:
        tags = elem.get('tags', {})
        name = tags.get('name', None)
        amenity = tags.get('amenity', 'n/a')
        if name is None:
            continue
        item = Item(f'{name} ({amenity})')
        item += get_osm_data(tags)
        if 'wikipedia' in tags:
            item += get_wiki_data(*tags['wikipedia'].split(':'))
        if 'website' in tags:
            item += get_website_data(tags['website'])

        items.append(item)

        if len(items) >= max_items:
            break
    
    for item in tqdm(items):
        await item.await_datasources()

else:
    print("Error:", response.status_code, response.text)

In [ ]:
for item in items:
    print(item)